<a href="https://colab.research.google.com/github/dimna21/ML_Assignment4/blob/main/FER2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install wandb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1) Load the CSV
csv_path = "/content/drive/MyDrive/FER_data/fer2013/fer2013.csv"
df = pd.read_csv(csv_path)

# 2) Split by Usage
df_train = df[df['Usage']=="Training"].copy()
df_val   = df[df['Usage']=="PublicTest"].copy()
df_test  = df[df['Usage']=="PrivateTest"].copy()

In [15]:
# 3) Balance function: upsample & random ±10 intensity shifts
def balance_dataset(df, target_count, img_shape=(48,48)):
    def augment(pix_str):
        arr = np.fromstring(pix_str, sep=' ', dtype=int).reshape(img_shape)
        shift = np.random.randint(-10, 11)
        arr = np.clip(arr + shift, 0, 255).astype(int)
        return ' '.join(map(str, arr.ravel()))
    parts = [df]
    for emo, grp in df.groupby('emotion'):
        n = len(grp)
        if n < target_count:
            extra = grp.sample(n=target_count-n, replace=True).copy()
            extra['pixels'] = extra['pixels'].map(augment)
            parts.append(extra)
    return pd.concat(parts, ignore_index=True)

max_count = df_train['emotion'].value_counts().max()
balanced_train = balance_dataset(df_train, target_count=max_count)

In [16]:
# 4) Dataset class
class FERDataset(Dataset):
    def __init__(self, dataframe):
        self.pixels = dataframe['pixels'].values
        self.labels = dataframe['emotion'].values.astype(int)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        arr = np.fromstring(self.pixels[idx], sep=' ', dtype=np.uint8).reshape(48,48)
        arr = arr.astype(np.float32) / 255.0
        tensor = torch.from_numpy(arr).unsqueeze(0)  # shape [1,48,48]
        return tensor, self.labels[idx]


In [17]:
# 5) Simple conv-based net: 5×Conv3×3 → Dropout → FC → FC
class BaselineModel(nn.Module):
    def __init__(self,
                 in_channels: int = 1,
                 conv_channels: int = 32,
                 hidden_dim: int    = 256,
                 num_classes: int   = 7,
                 dropout_p: float   = 0.5):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels,   conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(conv_channels, conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(conv_channels, conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.dropout = nn.Dropout(p=dropout_p)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(conv_channels * 48 * 48, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.convs(x)          # [B, conv_channels, 48, 48]
        x = self.dropout(x)        # dropout on feature maps
        x = self.flatten(x)        # [B, conv_channels*48*48]
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)            # [B, num_classes]
        return x

# Example instantiation
model = BaselineModel()
print(model)

BaselineModel(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=73728, out_features=256, bias=True)
  (relu): ReLU(inplace=True)
  (fc2): Linear(in_features=256, out_features=7, bias=True)
)


In [18]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [19]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# DataLoaders
batch_size = 128
train_ds = FERDataset(balanced_train)
val_ds   = FERDataset(df_val)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)


In [25]:
print(3)

3


In [10]:
import wandb
import torch
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

def train_model(
    model,
    train_loader,
    val_loader,
    test_loader,           # new: DataLoader for your held-out test set
    criterion,
    optimizer,
    device,
    epochs=5,
    class_names=None      # list of names for your classes
):
    wandb.init(
        project="ML_Assignment4",
        config={
            "epochs": epochs,
            "batch_size": train_loader.batch_size,
            "optimizer": optimizer.__class__.__name__,
            "lr": optimizer.param_groups[0]["lr"],
            "criterion": criterion.__class__.__name__,
        }
    )
    cfg = wandb.config
    wandb.watch(model, log="all", log_freq=100)

    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    model.to(device)

    for epoch in range(1, cfg.epochs + 1):
        # — TRAIN —
        model.train()
        tloss = 0.0
        tcorrect = 0
        for X, y in tqdm(train_loader, desc=f"[Train] epoch {epoch}"):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            tloss += loss.item() * X.size(0)
            tcorrect += (logits.argmax(1) == y).sum().item()
        train_loss = tloss / len(train_loader.dataset)
        train_acc  = tcorrect / len(train_loader.dataset)
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)

        # — VALIDATE —
        model.eval()
        vloss = 0.0
        vcorrect = 0
        with torch.no_grad():
            for X, y in tqdm(val_loader, desc=f"[Val] epoch {epoch}"):
                X, y = X.to(device), y.to(device)
                logits = model(X)
                loss = criterion(logits, y)
                vloss += loss.item() * X.size(0)
                vcorrect += (logits.argmax(1) == y).sum().item()
        val_loss = vloss / len(val_loader.dataset)
        val_acc  = vcorrect / len(val_loader.dataset)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Epoch {epoch}/{cfg.epochs} — "
              f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
              f"Val loss {val_loss:.4f}, acc {val_acc:.4f}")

        wandb.log({
            "epoch": epoch,
            "train/loss": train_loss,
            "train/accuracy": train_acc,
            "val/loss": val_loss,
            "val/accuracy": val_acc,
        })

    # — FINAL TEST EVAL —
    if test_loader is not None:
        model.eval()
        preds, targets = [], []
        with torch.no_grad():
            for X, y in tqdm(test_loader, desc="[Test]"):
                X = X.to(device)
                logits = model(X)
                preds.extend(logits.argmax(1).cpu().tolist())
                targets.extend(y.tolist())

        test_acc = accuracy_score(targets, preds)
        f1s = f1_score(targets, preds, average=None)
        cm = confusion_matrix(targets, preds)

        # log test accuracy
        wandb.log({"test/accuracy": test_acc})

        # log per-class F1
        for idx, cls in enumerate(class_names or map(str, range(len(f1s)))):
            wandb.log({f"test/f1_{cls}": f1s[idx]})

        # log confusion matrix
        wandb.log({
            "test/confusion_matrix": wandb.plot.confusion_matrix(
                probs=None,
                y_true=targets,
                preds=preds,
                class_names=list(class_names or map(str, range(len(f1s))))
            )
        })

        print(f"Test Acc: {test_acc:.4f}")
        print("Test F1 per class:", dict(zip(class_names or range(len(f1s)), f1s)))
        print("Confusion matrix:\n", cm)

    wandb.finish()
    return history


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BaselineModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
history = train_model(model, train_dl, val_dl, criterion, optimizer, device, epochs=5)

Epoch 1 [Val]: 100%|██████████| 29/29 [00:00<00:00, 62.27it/s]


Epoch 1/5 — Train loss: 1.5308, acc: 0.4079 | Val loss: 1.4808, acc: 0.4269


Epoch 2 [Val]: 100%|██████████| 29/29 [00:00<00:00, 58.72it/s]


Epoch 2/5 — Train loss: 0.9942, acc: 0.6308 | Val loss: 1.4076, acc: 0.4820


Epoch 3 [Val]: 100%|██████████| 29/29 [00:00<00:00, 63.36it/s]


Epoch 3/5 — Train loss: 0.6299, acc: 0.7796 | Val loss: 1.6394, acc: 0.4667


Epoch 4 [Val]: 100%|██████████| 29/29 [00:00<00:00, 60.77it/s]


Epoch 4/5 — Train loss: 0.3615, acc: 0.8765 | Val loss: 1.8953, acc: 0.4884


Epoch 5 [Val]: 100%|██████████| 29/29 [00:00<00:00, 39.82it/s]

Epoch 5/5 — Train loss: 0.2114, acc: 0.9306 | Val loss: 2.3858, acc: 0.4865
